This notebook is an (in progress) example of kernel optimization using SVI in gp3. Use NBViewer to view the notebook [here](https://nbviewer.jupyter.org/github/as4529/gp3/blob/master/examples/kernel_learning.ipynb?flush_cache=true). 

In [1]:
import sys
sys.path.insert(0,'..')
from gp3.inference.mfsvi import MFSVI
from gp3.inference.laplace import Laplace
from gp3.likelihoods.likelihoods import Poisson, Bernoulli
from gp3.utils import data as sim
from gp3.kernels.kernels import rbf, softmax, inv_softmax
from gp3.utils.kron import kron_list_diag, kron_mvp
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
from tqdm import trange
import numpy as np
from scipy.special import expit

First, we simulate some data from a GP with an RBF kernel.

In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 20, lower=0, upper=100)
f = sim.sim_f(X, rbf, inv_softmax(np.array([20., 1.0])), mu = 4.)
y = sim.poisson_draw(f, 0.5) 

trace_func = go.Scatter3d(x = X[:,0], y = X[:,1], z=f, mode = 'markers', marker=dict(size = 2,))
trace_draws = go.Scatter3d(x = X[:,0], y = X[:,1], z=y, mode = 'markers', marker=dict(size = 2,))
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_draws, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



Now, we set the "opt_kernel" parameter for the mean field SVI inference to True, so that we optimize over our base kernel hyperparameters as we optimize the ELBO. This is currently very sensitive to initialization. Addressing this sensitivity is my current priority. Also note that (as expected) there is around a 5x slowdown when we are learning the kernel as opposed to keeping it fixed.

In [3]:
inf_svi = MFSVI(rbf, inv_softmax(np.array([1., 1.])), Poisson(), X, y, opt_kernel = True, noise = 0.5)
inf_svi.run(500)

ELBO: 171974.84 | KL: 1397.78 | logL: 173372.62: 100%|██████████| 500/500 [00:20<00:00, 24.06it/s]


In [4]:
softmax(inf_svi.kernel_params)

array([ 7.09378277,  2.18662414])

Plotting posterior mean and variances

In [5]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=inf_svi.predict(), mode = 'markers', marker=dict(size = 2,))])
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=np.exp(inf_svi.q_S), mode = 'markers', marker=dict(size = 2,))])

We can take a look at the convergence of the ELBO, KL, and likelihood.

In [6]:
elbos = [e[0] for e in inf_svi.objs]
kls = [e[1] for e in inf_svi.objs]
likes = [e[2] for e in inf_svi.objs]

iplot([go.Scatter(x = range(len(elbos)), y = elbos)])
iplot([go.Scatter(x = range(len(likes)), y = likes)])
iplot([go.Scatter(x = range(len(kls)), y = kls)])